In [75]:
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import precision_score,recall_score, confusion_matrix, classification_report,accuracy_score, f1_score
import numpy as np
import pandas as pd
import random
from sklearn import metrics
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve , auc
from sklearn.metrics.cluster import fowlkes_mallows_score as gmean
import statistics as stat

In [76]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [77]:
df=pd.read_csv('/content/drive/MyDrive/yeast-1_vs_7.csv')
df

,Mcg,Gvh,Alm,Mit,Erl,Vac,Nuc,Class
0,0.58,0.44,0.57,0.13,0.5,0.54,0.22,negative
1,0.48,0.45,0.59,0.20,0.5,0.58,0.34,negative
2,0.43,0.39,0.54,0.21,0.5,0.53,0.27,negative
3,0.42,0.37,0.59,0.20,0.5,0.52,0.29,negative
4,0.58,0.47,0.54,0.11,0.5,0.51,0.26,negative
...,...,...,...,...,...,...,...,...
454,0.65,0.58,0.53,0.21,0.5,0.51,0.22,positive
455,0.48,0.49,0.49,0.26,0.5,0.39,0.27,positive
456,0.57,0.54,0.32,0.15,0.5,0.52,0.25,positive
457,0.40,0.36,0.32,0.16,0.5,0.51,0.27,positive


In [78]:
df["Class"].value_counts()
class_label=df['Class']
class_label

0      negative
1      negative
2      negative
3      negative
4      negative
         ...   
454    positive
455    positive
456    positive
457    positive
458    positive
Name: Class, Length: 459, dtype: object

In [79]:
label_encoder = LabelEncoder()
df["Class"] = label_encoder.fit_transform(df["Class"])
df.head()


,Mcg,Gvh,Alm,Mit,Erl,Vac,Nuc,Class
0,0.58,0.44,0.57,0.13,0.5,0.54,0.22,0
1,0.48,0.45,0.59,0.20,0.5,0.58,0.34,0
2,0.43,0.39,0.54,0.21,0.5,0.53,0.27,0
3,0.42,0.37,0.59,0.20,0.5,0.52,0.29,0
4,0.58,0.47,0.54,0.11,0.5,0.51,0.26,0


In [80]:
minor=df[df["Class"]==1]
major=df[df["Class"]==0]
minor=minor.drop(["Class"], axis=1)
minority=minor.to_numpy()
minor.shape
major.shape
diff=major.shape[0]-minor.shape[0]
r=diff
loop=diff/minor.shape[0]
if(diff%minor.shape[0]==0):
    iter=int(diff/minor.shape[0])
else:
  iter=int(diff/minor.shape[0])+1
minor.shape


(30, 7)

In [81]:
for id in range(r):
  if(diff>0):
    distances = np.linalg.norm(minority - minority[id], axis=1)  #calculating the distances to other instances
    k = 4
    nearest_neighbor_ids = distances.argsort()[:k] #finding 3 nearest neighbours n1,n2 and n3
    n1=minority[nearest_neighbor_ids[1]]
    n2=minority[nearest_neighbor_ids[2]]
    n3=minority[nearest_neighbor_ids[3]]
    r1=random.uniform(0,1)
    r2=random.uniform(0,1)
    w=random.uniform(0.5,1)
    velocity= w+r1*(n1-n3) + r2*(n1-n2)         #velocity vector
    new_minor=minority[id]+velocity             #position update
    d=minor.shape[1] 
    minB=minor.min().to_numpy()
    maxB=minor.max().to_numpy()
    for i in range(d):
      if(new_minor[i]<minB[i]):
        new_minor[i]=minB[i];
      if(new_minor[i]>maxB[i]):
        new_minor[i]=maxB[i];
    minority=np.append(minority,new_minor)
    minority=np.reshape(minority,(-1, minor.shape[1]))
    diff=diff-1

minority.shape

(429, 7)

In [82]:
df_minor = pd.DataFrame(minority, columns =['Mcg','Gvh','Alm','Mit','Erl','Vac','Nuc'])
df_minor

,Mcg,Gvh,Alm,Mit,Erl,Vac,Nuc
0,0.37,0.36,0.56,0.18,0.5,0.48,0.26
1,0.74,0.57,0.51,0.28,0.5,0.58,0.22
2,0.70,0.65,0.54,0.28,0.5,0.60,0.40
3,0.36,0.68,0.38,0.08,0.5,0.53,0.25
4,0.42,0.32,0.48,0.16,0.5,0.55,0.22
...,...,...,...,...,...,...,...
424,0.80,0.82,0.58,0.51,0.5,0.62,0.40
425,0.80,0.82,0.58,0.51,0.5,0.62,0.40
426,0.80,0.82,0.58,0.51,0.5,0.62,0.40
427,0.80,0.82,0.58,0.51,0.5,0.62,0.40


In [83]:
dup=[]
for i in range(df_minor.shape[0]):
  dup.append(1)
Class=np.array(dup)
Class=np.reshape(Class,(-1,1))
df_minor_class=pd.DataFrame(Class,columns=['Class'])
df_minor['Class']=df_minor_class
df_minor

,Mcg,Gvh,Alm,Mit,Erl,Vac,Nuc,Class
0,0.37,0.36,0.56,0.18,0.5,0.48,0.26,1
1,0.74,0.57,0.51,0.28,0.5,0.58,0.22,1
2,0.70,0.65,0.54,0.28,0.5,0.60,0.40,1
3,0.36,0.68,0.38,0.08,0.5,0.53,0.25,1
4,0.42,0.32,0.48,0.16,0.5,0.55,0.22,1
...,...,...,...,...,...,...,...,...
424,0.80,0.82,0.58,0.51,0.5,0.62,0.40,1
425,0.80,0.82,0.58,0.51,0.5,0.62,0.40,1
426,0.80,0.82,0.58,0.51,0.5,0.62,0.40,1
427,0.80,0.82,0.58,0.51,0.5,0.62,0.40,1


In [84]:
majority=major.to_numpy()
majority=np.reshape(majority,(-1,major.shape[1]))
majority
df_major = pd.DataFrame(majority, columns = ['Mcg','Gvh','Alm','Mit','Erl','Vac','Nuc','Class'])
df_major

,Mcg,Gvh,Alm,Mit,Erl,Vac,Nuc,Class
0,0.58,0.44,0.57,0.13,0.5,0.54,0.22,0.0
1,0.48,0.45,0.59,0.20,0.5,0.58,0.34,0.0
2,0.43,0.39,0.54,0.21,0.5,0.53,0.27,0.0
3,0.42,0.37,0.59,0.20,0.5,0.52,0.29,0.0
4,0.58,0.47,0.54,0.11,0.5,0.51,0.26,0.0
...,...,...,...,...,...,...,...,...
424,0.25,0.40,0.52,0.15,0.5,0.56,0.28,0.0
425,0.58,0.56,0.38,0.39,0.5,0.54,0.57,0.0
426,0.59,0.67,0.54,0.19,0.5,0.48,0.60,0.0
427,0.47,0.43,0.61,0.40,0.5,0.48,0.47,0.0


In [85]:
balanced_df=pd.concat([df_minor,df_major],axis=0,ignore_index=True)
balanced_df

,Mcg,Gvh,Alm,Mit,Erl,Vac,Nuc,Class
0,0.37,0.36,0.56,0.18,0.5,0.48,0.26,1.0
1,0.74,0.57,0.51,0.28,0.5,0.58,0.22,1.0
2,0.70,0.65,0.54,0.28,0.5,0.60,0.40,1.0
3,0.36,0.68,0.38,0.08,0.5,0.53,0.25,1.0
4,0.42,0.32,0.48,0.16,0.5,0.55,0.22,1.0
...,...,...,...,...,...,...,...,...
853,0.25,0.40,0.52,0.15,0.5,0.56,0.28,0.0
854,0.58,0.56,0.38,0.39,0.5,0.54,0.57,0.0
855,0.59,0.67,0.54,0.19,0.5,0.48,0.60,0.0
856,0.47,0.43,0.61,0.40,0.5,0.48,0.47,0.0


In [86]:

from sklearn.model_selection import StratifiedKFold
kf=StratifiedKFold(n_splits=5)
x_b=balanced_df.drop(['Class'],axis='columns')
y_b=balanced_df.Class




In [87]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
model_b_s=SVC()

model_b_k=KNeighborsClassifier(n_neighbors=3)


model_b_d=DecisionTreeClassifier(random_state=0)

In [88]:
#precision
precision_s=[]
f1_s=[]
recall_s=[]
accuracy_s=[]
Auc_s=[]
Gmean_s=[]

precision_k=[]
f1_k=[]
recall_k=[]
accuracy_k=[]
Auc_k=[]
Gmean_k=[]

precision_d=[]
f1_d=[]
recall_d=[]
accuracy_d=[]
Auc_d=[]
Gmean_d=[]
for train_index_b,test_index_b in kf.split(x_b,y_b):
  x_train_b,x_test_b=x_b.iloc[train_index_b],x_b.iloc[test_index_b]
  y_train_b,y_test_b=y_b.iloc[train_index_b],y_b.iloc[test_index_b]
  model_b_s.fit(x_train_b,y_train_b)
  prediction_b_s = model_b_s.predict(x_test_b)

  model_b_k.fit(x_train_b,y_train_b)
  prediction_b_k = model_b_k.predict(x_test_b)

  model_b_d.fit(x_train_b,y_train_b)
  prediction_b_d = model_b_d.predict(x_test_b)

  precision_s.append(precision_score(y_test_b,prediction_b_s))
  precision_k.append(precision_score(y_test_b,prediction_b_k))
  precision_d.append(precision_score(y_test_b,prediction_b_d))

  recall_s.append(recall_score(y_test_b,prediction_b_s))
  recall_k.append(recall_score(y_test_b,prediction_b_k))
  recall_d.append(recall_score(y_test_b,prediction_b_d))

  f1_s.append(f1_score(y_test_b,prediction_b_s))
  f1_k.append(f1_score(y_test_b,prediction_b_k))
  f1_d.append(f1_score(y_test_b,prediction_b_d))

  accuracy_s.append(accuracy_score(y_test_b,prediction_b_s))
  accuracy_k.append(accuracy_score(y_test_b,prediction_b_k))
  accuracy_d.append(accuracy_score(y_test_b,prediction_b_d))


  s_fpr_b,s_tpr_b,threshold_s_b=roc_curve(y_test_b,prediction_b_s)
  k_fpr_b,k_tpr_b,threshold_k_b=roc_curve(y_test_b,prediction_b_k)
  d_fpr_b,d_tpr_b,threshold_d_b=roc_curve(y_test_b,prediction_b_d)


  Auc_s.append(auc(s_fpr_b,s_tpr_b))
  Auc_k.append(auc(k_fpr_b,k_tpr_b))
  Auc_d.append(auc(d_fpr_b,d_tpr_b))


  Gmean_s.append(gmean(y_test_b,prediction_b_s))
  Gmean_k.append(gmean(y_test_b,prediction_b_k))
  Gmean_d.append(gmean(y_test_b,prediction_b_d))


print("-------SVM--------")
print("Gmean_SVM:",Gmean_s)
print("AUC_SVM:",Auc_s)
print("f1_SVM:",f1_s)
print("precision_SVM:",precision_s)
print("recall_SVM:",recall_s)

print("mean of Gmean_s:",stat.mean(Gmean_s))
print("mean of AUC_s:",stat.mean(Auc_s))
print("mean of f1_s:",stat.mean(f1_s))
print("mean of precision_s:",stat.mean(precision_s))
print("mean of recall_s:",stat.mean(recall_s))

print("standard deviation of Gmean_s:",stat.stdev(Gmean_s))
print("standard deviation of AUC_s:",stat.stdev(Auc_s))
print("standard deviation of f1_s:",stat.stdev(f1_s))
print("standard deviation of precision_s:",stat.stdev(precision_s))
print("standard deviation of recall_s:",stat.stdev(recall_s))

print("\n")
print("-------KNN--------")
print("Gmean_KNN:",Gmean_k)
print("AUC_KNN:",Auc_k)
print("f1_KNN:",f1_k)
print("precision_KNN:",precision_k)
print("recall_KNN:",recall_k)

print("mean of Gmean_k:",stat.mean(Gmean_k))
print("mean of AUC_k:",stat.mean(Auc_k))
print("mean of f1_k:",stat.mean(f1_k))
print("mean of precision_k:",stat.mean(precision_k))
print("mean of recall_k:",stat.mean(recall_k))

print("standard deviation of Gmean_k:",stat.stdev(Gmean_k))
print("standard deviation of AUC_k:",stat.stdev(Auc_k))
print("standard deviation of f1_k:",stat.stdev(f1_k))
print("standard deviation of precision_k:",stat.stdev(precision_k))
print("standard deviation of recall_k:",stat.stdev(recall_k))

print("\n")
print("-------DECISION TREE--------")
print("Gmean_DT:",Gmean_d)
print("AUC_DT:",Auc_d)
print("f1_DT:",f1_d)
print("precision_DT:",precision_d)
print("recall_DT:",recall_d)

print("mean of Gmean_d:",stat.mean(Gmean_d))
print("mean of AUC_d:",stat.mean(Auc_d))
print("mean of f1_d:",stat.mean(f1_d))
print("mean of precision_d:",stat.mean(precision_d))
print("mean of recall_d:",stat.mean(recall_d))

print("standard deviation of Gmean_d:",stat.stdev(Gmean_d))
print("standard deviation of AUC_d:",stat.stdev(Auc_d))
print("standard deviation of f1_d:",stat.stdev(f1_d))
print("standard deviation of precision_d:",stat.stdev(precision_d))
print("standard deviation of recall_d:",stat.stdev(recall_d))



-------SVM--------
Gmean_SVM: [0.7183521822721247, 1.0, 1.0, 1.0, 1.0]
AUC_SVM: [0.8197674418604651, 1.0, 1.0, 1.0, 1.0]
f1_SVM: [0.7832167832167831, 1.0, 1.0, 1.0, 1.0]
precision_SVM: [0.9824561403508771, 1.0, 1.0, 1.0, 1.0]
recall_SVM: [0.6511627906976745, 1.0, 1.0, 1.0, 1.0]
mean of Gmean_s: 0.9436704364544249
mean of AUC_s: 0.963953488372093
mean of f1_s: 0.9566433566433566
mean of precision_s: 0.9964912280701754
mean of recall_s: 0.9302325581395349
standard deviation of Gmean_s: 0.1259567332307999
standard deviation of AUC_s: 0.0806024503517366
standard deviation of f1_s: 0.09694840182166925
standard deviation of precision_s: 0.007845852552630865
standard deviation of recall_s: 0.15600474261626437


-------KNN--------
Gmean_KNN: [0.7183521822721247, 0.9883044958482615, 0.9320442964959241, 0.9882369316884356, 0.988235294117647]
AUC_KNN: [0.8197674418604651, 0.9941860465116279, 0.9651162790697674, 0.9941176470588236, 0.9941860465116279]
f1_KNN: [0.7832167832167831, 0.994219653179190

In [91]:
list=['yeast1vs7',stat.mean(Auc_k),stat.stdev(Auc_k),stat.mean(Gmean_k),stat.stdev(Gmean_k),stat.mean(Auc_d),stat.stdev(Auc_d),stat.mean(Gmean_d),stat.stdev(Gmean_d),stat.mean(Auc_s),stat.stdev(Auc_s),stat.mean(Gmean_s),stat.stdev(Gmean_s)]
list

df_excel = pd.read_excel('/content/drive/MyDrive/pso_1.xlsx')
df_excel

,Unnamed: 0,Unnamed: 0.1,KNN,Unnamed: 2,Unnamed: 3,Unnamed: 4,Decision Tree,Unnamed: 6,Unnamed: 7,Unnamed: 8,SVM,Unnamed: 10,Unnamed: 11,Unnamed: 12
0,0,NaN,AUC,NaN,Gmean,NaN,AUC,NaN,Gmean,NaN,AUC,NaN,Gmean,NaN
1,1,Dataset,mean,st.dev,mean,st.dev,mean,st.dev,mean,st.dev,mean,st.dev,mean,st.dev
2,2,ecoli0137vs26,0.959495,0.040136,0.92449,0.072456,0.959798,0.027859,0.923228,0.052104,0.946633,0.061364,0.905228,0.105836
3,3,shuttle0vs4,0.999414,0.000802,0.998826,0.001607,1,0,1,0,0.998828,0.000655,0.997651,0.001313


In [92]:
df_excel.loc[len(df_excel)]=list
df_excel.to_excel('/content/drive/MyDrive/pso_1.xlsx')